In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [4]:
df = pd.read_csv('../input/ams-prediction/amsPredictionSheet11.csv')
df.head()

,Attendance,MSE,HRS,ESE
0,70,10,17,42
1,92,7,20,39
2,67,3,18,32
3,82,16,13,50
4,80,9,10,44


In [5]:
df.describe()

,Attendance,MSE,HRS,ESE
count,73.000000,73.000000,73.000000,73.000000
mean,81.698630,9.780822,12.904110,46.630137
std,9.506847,3.047069,5.701278,6.617542
min,62.000000,3.000000,2.000000,32.000000
25%,75.000000,8.000000,8.000000,42.000000
50%,80.000000,10.000000,14.000000,47.000000
75%,90.000000,11.000000,18.000000,50.000000
max,98.000000,17.000000,22.000000,64.000000


In [6]:
endog = df['ESE'].head(4)
exog =  df[['MSE','Attendance','HRS']].head(4)
print(exog)

   MSE  Attendance  HRS
0   10          70   17
1    7          92   20
2    3          67   18
3   16          82   13


In [7]:
X=exog.to_numpy()
Y= endog.to_numpy()
print(X)

[[10 70 17]
 [ 7 92 20]
 [ 3 67 18]
 [16 82 13]]


In [8]:
from numpy import array
from numpy import mean
from numpy import cov
from numpy.linalg import eig

In [9]:
M = mean(X.T, axis=1)
print(M)

[ 9.   77.75 17.  ]


In [10]:
C= X-M
print(C)

[[  1.    -7.75   0.  ]
 [ -2.    14.25   3.  ]
 [ -6.   -10.75   1.  ]
 [  7.     4.25  -4.  ]]


In [11]:
V = cov(C.T)
print(V)

[[ 30.          19.33333333 -13.33333333]
 [ 19.33333333 132.25         5.        ]
 [-13.33333333   5.           8.66666667]]


In [12]:
values, vectors = eig(V)
print(vectors)
print(values)

[[-0.17721376 -0.86455713  0.47025126]
 [-0.98396696  0.14588088 -0.10260501]
 [-0.02010722  0.48089472  0.87654776]]
[135.8341334   34.15422626   0.928307  ]


In [13]:
# project data
P = vectors.T.dot(C.T)
print(P.T)

[[  7.44853019  -1.99513394   1.26544005]
 [-13.72742335   5.25060095   0.22701942]
 [ 11.62082019   4.10001808  -0.84195598]
 [ -5.34192703  -7.35548508  -0.6505035 ]]


In [15]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
principalComponents = pca.fit_transform(X)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1'])

In [16]:
exog1 = sm.add_constant(principalDf[['principal component 1']])
print(exog1)

   const  principal component 1
0    1.0              -7.448530
1    1.0              13.727423
2    1.0             -11.620820
3    1.0               5.341927


In [18]:
from sklearn import linear_model
lm = linear_model.LinearRegression()
model = lm.fit(exog1,endog)
lm.coef_

array([0.       , 0.2889832])

In [19]:
lm.intercept_

40.75

In [20]:
def RSE(y_true, y_predicted):
   
    y_true = np.array(y_true)
    y_predicted = np.array(y_predicted)
    RSS = np.sum(np.square(y_true - y_predicted))

    rse = math.sqrt(RSS / (len(y_true) - 2))
    return rse

In [21]:
ypred =lm.predict(exog1)

In [ ]:
re = RSE(endog,ypred)
print(re)